# 라이브러리

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

# 데이터 불러오기 및 처리
- head로 dateframe 모양 확인
- folium으로 시각화 할 예정이므로 epsg 확인 및 변환
- 주택만 시각화할 것이므로 필터링
- polygon 정보를 이용해 point(x,y) 추출

In [2]:
gdf = gpd.read_file('../../Data_list/result/건물공간_데이터/용도별건물공간확보_부산_가공/수영구/수영구.shp')

In [3]:
gdf.head(2)

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A27,A28,A29,A30,A31,A32,A33,A34,A35,geometry
0,34041161,1991210573681863652700000000,2650010100101090004,2650010100,부산광역시 수영구 망미동,1,일반,109-4,104111143,1,...,단독주택,1,주거용,7.8,2.0,0.0,1990-08-21,1991-03-13,2025-01-10 02:26:32.096283,"POLYGON ((392824.671 288723.284, 392834.636 28..."
1,31847756,1989210573601863758400000000,2650010100101090005,2650010100,부산광역시 수영구 망미동,1,일반,109-5,104111144,1,...,다가구주택,1,주거용,0.0,2.0,0.0,1989-01-24,1989-04-07,2025-01-10 02:26:32.096283,"POLYGON ((392833.685 288735.303, 392834.64 288..."


In [4]:
gdf = gdf.to_crs(epsg=4326)
print(gdf.crs.to_epsg())


4326


In [5]:
display(gdf['A25'].value_counts())
display(gdf['A27'].value_counts())
display(gdf['A29'].value_counts())

A25
단독주택         8100
공동주택         2076
제2종근린생활시설     145
교육연구시설         44
노유자시설          43
제1종근린생활시설       4
업무시설            2
숙박시설            1
판매시설            1
Name: count, dtype: int64

A27
단독주택        6396
다가구주택       1695
다세대주택       1559
아파트          514
학원           175
영유아보육시설       17
유치원           12
어린이집          11
다중주택          11
중학교            8
초등학교           7
공동주택           5
기타아동관련시설       4
노유자시설          1
아동복지시설         1
주거용            1
Name: count, dtype: int64

A29
주거용      10125
상업용        201
문교사회용       90
Name: count, dtype: int64

In [6]:
gdf = gdf[gdf['A25'].str.contains('주택', na=False)]
gdf['A25'].value_counts()

A25
단독주택    8100
공동주택    2076
Name: count, dtype: int64

In [7]:
gdf['dot'] = gdf['geometry'].apply(lambda x: x.centroid)
gdf.iloc[:, -2:].head(2)

,geometry,dot
0,"POLYGON ((129.11665 35.17645, 129.11676 35.176...",POINT (129.11671 35.17642)
1,"POLYGON ((129.11676 35.17656, 129.11676 35.176...",POINT (129.11671 35.17652)


In [8]:
print(gdf['dot'].isna().sum())

0


# 시각화 - polygon과 point
- 초기: 10개만 시각화하여 제대로 생성되는지 확인


In [9]:
# 시각화 라이브러리
import folium

In [10]:
# 샘플 데이터셋 생성 (10개만)
gdf_sample = gdf.head(10)

# 샘플 데이터셋의 중심 좌표 계산
mean_lat = gdf_sample['dot'].apply(lambda p: p.y).mean()
mean_lon = gdf_sample['dot'].apply(lambda p: p.x).mean()

# 중심 좌표를 기준으로 지도를 생성
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=36)

# 샘플 데이터셋의 지오메트리를 지도에 추가
folium.GeoJson(gdf_sample[['geometry']].to_json()).add_to(m)

# 각 샘플 데이터셋의 중심점에 원을 추가
for idx, row in gdf_sample.iterrows():
    if row['dot'] is not None:
        folium.Circle(
            location=[row['dot'].y, row['dot'].x],  # 중심점의 좌표
            radius=1,  # 원의 반지름
            color='red',  # 원의 선을 설정
            fill=True,  # 원을 채워넣을지 여부
            fill_color='red'  # 원을 채워넣을 때의 색
        ).add_to(m)

# 생성된 지도를 표시
m

# 지오메트리와 포인트를 따로 나눠서 찍는 이유:
# 한 번에 처리 시 TypeError: Object of type Point is not JSON serializable 오류 발생

In [18]:
# MeanShift 구현
from sklearn.cluster import MeanShift

# 대역폭 설정 (20m를 도 단위로 변환, 1도 ≈ 111km)
bandwidth = 0.0022  # 약 20m에 해당

# MeanShift 모델 생성 및 학습
model = MeanShift(bandwidth=bandwidth, bin_seeding=True)
model.fit(coords)
clusters = model.labels_

gdf['cluster'] = clusters

# 클러스터 중심점 계산
optimal_locations = {}
for cluster_id in np.unique(clusters):
    if cluster_id != -1:
        center_lon, center_lat = model.cluster_centers_[cluster_id]
        optimal_locations[cluster_id] = (center_lat, center_lon)

# 시각화 부분은 기존 코드 유지
colors = plt.cm.tab10.colors
color_list = [f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}' for r, g, b in colors]

# 중심점 좌표 설정
if optimal_locations:
    first_cluster_id = list(optimal_locations.keys())[0]
    center_lat, center_lon = optimal_locations[first_cluster_id]
elif not gdf.empty:
    center_lat = gdf['dot'].iloc[0].y
    center_lon = gdf['dot'].iloc[0].x
else:
    center_lat = 35.17
    center_lon = 129.12

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# 클러스터 포인트 시각화
for idx, row in gdf.iterrows():
    lat = row['dot'].y
    lon = row['dot'].x
    cluster = row['cluster']
    
    color = color_list[cluster % len(color_list)]  # 모든 포인트가 클러스터에 할당됨
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=3,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8,
        popup=f'cluster: {cluster}'
    ).add_to(m)

# 최적 위치 마커 추가
for cluster_id, loc in optimal_locations.items():
    folium.Marker(
        location=[loc[0], loc[1]],
        icon=folium.Icon(color='red', icon='info-sign'),
        popup=f'Optimal Location for Cluster {cluster_id}'
    ).add_to(m)

m.save('meanshift_clusters_and_optimal_locations.html')